<a href="https://colab.research.google.com/github/dschomme/ADS/blob/main/ADS_12%EC%A3%BC%EC%B0%A8%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 심리성향 예측 데이터 분류모델

# 데이터 변수 설명

index
Q_A / Q_E (a~t) 비식별화를 위해 일부 질문은 Secret 처리
Qa : Secret
Qb : The biggest difference between most criminals and other people is that the criminals are stupid enough to get caught.
Qc : Anyone who completely trusts anyone else is asking for trouble.
Qd : Secret
Qe : P.T. Barnum was wrong when he said that there's a sucker born every minute.
Qf : There is no excuse for lying to someone else.
Qg : Secret
Qh : Most people forget more easily the death of their parents than the loss of their property.
Qi : Secret
Qj : It is safest to assume that all people have a vicious streak and it will come out when they are given a chance.
Qk : All in all, it is better to be humble and honest than to be important and dishonest.
Ql : Secret
Qm : It is hard to get ahead without cutting corners here and there.
Qn : Secret
Qo : The best way to handle people is to tell them what they want to hear.
Qp : Secret
Qq : Most people are basically good and kind.
Qr : One should take action only when sure it is morally right.
Qs : It is wise to flatter important people.
Qt : Secret


					1=Disagree, 2=Slightly disagree, 3=Neutral, 4=Slightly agree, 5=Agree.



Q_E(a~t) : 질문을 답할 때까지의 시간


age_group : 연령


education : 교육 수준
1=Less than high school, 2=High school, 3=University degree, 4=Graduate degree, 0=무응답


engnat : 모국어가 영어
1=Yes, 2=No, 0=무응답


familysize : 형제자매 수


gender : 성별
Male, Female


hand : 필기하는 손
1=Right, 2=Left, 3=Both, 0=무응답


married : 혼인 상태
1=Never married, 2=Currently married, 3=Previously married, 0=Other


race : 인종
Asian, Arab, Black, Indigenous Australian, Native American, White, Other


religion : 종교
Agnostic, Atheist, Buddhist, Christian_Catholic, Christian_Mormon, Christian_Protestant, Christian_Other, Hindu, Jewish, Muslim, Sikh, Other


tp__(01~07) : items were rated "I see myself as:" _____ such that
tp01 : Extraverted, enthusiastic.
tp02 : Critical, quarrelsome.
tp03 : Dependable, self-disciplined.
tp04 : Anxious, easily upset.
tp05 : Open to new experiences, complex.
tp06 : Reserved, quiet.
tp07 : Sympathetic, warm.
tp08 : Disorganized, careless.
tp09 : Calm, emotionally stable.
tp10 : Conventional, uncreative.
								

								순서형 변수

							



urban : 유년기의 거주 구역
1=Rural (country side), 2=Suburban, 3=Urban (town, city), 0=무응답




wr_(01~13) : 실존하는 해당 단어의 정의을 앎
1=Yes, 0=No
wf_(01~03) : 허구인 단어의 정의를 앎
1=Yes, 0=No


voted (타겟): 지난 해 국가 선거 투표 여부
1=Yes, 2=No


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from string import ascii_lowercase
from itertools import combinations

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = '/content/drive/MyDrive/dataset/train.csv'
test = '/content/drive/MyDrive/dataset/test_x.csv'
submission = '/content/drive/MyDrive/dataset/sample_submission.csv'

In [ ]:
train = pd.read_csv(train)
test = pd.read_csv(test)

In [ ]:
train

In [ ]:
train.isnull().sum()

각 질문에 대한 답변과, 그 답변의 시간으로 새로운 데이터 셋을 만들고 이들을 비슷한 집단끼리 군집화하여 비교를 해보자.

### 파생변수 생성

In [ ]:
eda_train = train.copy()

In [ ]:
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

In [ ]:
correlations = eda_train[Answers].corr(method = 'spearman')
sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

위의 질문들이 마키아벨라즘 테스트 임을 알 수 있음 -> 마키아벨리즘 테스트 파생변수 생성

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wlgid0727&logNo=220723273746

마키아벨리니즘 성향이 높은 사람들이 있다면 + 부호인 문항들에서는 높은 답변값을 내놓고, -부호인 문항들에서는 낮은 답변값을 내놓았을 것으로 판단. => pearson 상관계수가 아닌 spearman 상관계수를 쓰는 근거가 되기도 함.
speraman 상관계수는 pearson과는 다르게 linear 가정이 아니라 monotonic 가정을 함

Pearson 상관은 두 계량형 변수 사이의 선형 관계를 평가합니다. Spearman 상관 계수는 원시 데이터가 아니라 각 변수에 대해 순위를 매긴 값을 기반으로 합니다. Spearman 상관은 종종 순서형 변수가 포함된 관계를 평가하기 위해 사용됩니다.

- 질문데이터는 순서형 데이터

=> 질문에 대한 답변의 직관적인 해석을 위해 -부호의 답변값을 reverse 처리

In [ ]:
for col in eda_train[Answers]:
    print(sorted(eda_train[col].unique()))

모든 문항의 답변이 1, 2, 3, 4, 5로 되어있기 때문에 답변값을 reverse시키기 위해서는 단순히 6에서 답변값을 빼주면 됩니다.

In [ ]:
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    eda_train[flip] = 6 - eda_train[flip]

In [ ]:
correlations = eda_train[Answers].corr(method = 'spearman')
sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

In [ ]:
correlations = eda_train[Answers].corr(method = 'spearman')
sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

위의 음의 상관관계를 보이는 문항에서 -> 음의 부호의 문항임을 확인 -> reverse처리

In [ ]:
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    eda_train[flip] = 6 - eda_train[flip]

In [ ]:
correlations = eda_train[Answers].corr(method='spearman')
sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

전체 문항들을 +문항들의 직관적인 해석이 가능하게끔 변환함.

이러한 마키아벨리니즘 테스트를 기반으로 이를 지표로 나타낸 스코어를 파생변수를 생성함

In [ ]:
eda_train['Mach_score'] = eda_train[Answers].mean(axis=1)
eda_train.head()

In [ ]:
eda_train[['Mach_score']].describe()

In [ ]:
sns.displot(eda_train, x="Mach_score", bins=100)

In [ ]:
eda_train["Mach_score"].describe()

In [ ]:
eda_train['mach_size'] = eda_train['Mach_score'].apply(lambda x: 1 if x<=2.75 else (2 if x<=3.35 else (3 if x<=3.95 else 4)))

In [ ]:
eda_train

In [ ]:
questions = [i for i in list(ascii_lowercase)[:20]]
eda_train['delay'] = eda_train[[('Q'+i+'E') for i in questions]].sum(axis=1)

In [ ]:
eda_train['delay']

In [ ]:
tlist =pd.DataFrame([eda_train['Mach_score'], eda_train['delay']])

In [ ]:
#sns.heatmap(tlist.corr(), cmap='RdBu_r')

In [ ]:
tlist=tlist.transpose()

In [ ]:
sns.heatmap(tlist.corr(), cmap='RdBu_r')

상관성 없음

질문 변수들에 대하여 mean-encoding 진행

In [ ]:
QA_columns = list(eda_train.loc[:, eda_train.columns[1:41:2]].columns)
train_QA = eda_train.loc[:,QA_columns]
display(train_QA.head())

display(train_QA.describe())

In [ ]:
fig,ax = plt.subplots(5,4, figsize=(10,15))

for i in range(20):
    x_num = i//4
    y_num = i%4

    plot_dict = train_QA.iloc[:,i].value_counts().to_dict()
    x_values = sorted(plot_dict.keys())
    y_values = [plot_dict[x] for x in x_values]
    
    ax[x_num, y_num].bar(x_values, y_values)
    ax[x_num, y_num].set_title(train_QA.columns[i])
    ax[x_num, y_num].set_ylim(0,40000)
plt.tight_layout()
plt.show()